In [ ]:
from component import tile 
from component import io

In [ ]:
tb_io = io.TableIo()

In [ ]:
#create a test downloader
test_tile = tile.TestTile()

In [ ]:
# tiles to display the points on a map
tb_map_tile = tile.MapTile()

In [ ]:
#output to display messages
file_tile = tile.FileTile(tb_io, tb_map_tile.map)

In [ ]:
test_tile

In [ ]:
file_tile

In [ ]:
tb_map_tile

In [ ]:
tb_io.json_table